In [1]:
import langchain
import openai

In [2]:
from langchain_openai import OpenAI

In [3]:
import os

In [4]:
llm = OpenAI(
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    temperature = 0,
    model_name = "gpt-3.5-turbo-instruct"
)

In [5]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

In [62]:
llm_math = LLMMathChain(llm=llm)

In [7]:
math_tool = Tool(name='Calculator',func=llm_math.run, description='Useful when you need to answer questions about math.')
tools = [math_tool]

In [8]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [9]:
zero_shot_agent.invoke("What is 10^(-1)")

In [10]:
zero_shot_agent("""If Trenton has 7 candies and Georgio brings two and a half
candy boxes(each box contains 10 candies), how many candies do we have?""")

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables = ["query"],
    template = "{query}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [64]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables.base import RunnableSequence

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

# Assuming `llm` is your language model instance already created or imported
chain = prompt | llm


In [12]:
# add another tool to agent's tool box

In [13]:
llm_tool = Tool(
    name = "Language Model",
    func = llm_chain.run,
    description="use this tool for general purpose queries and logic"
)

In [14]:
tools.append(llm_tool)

In [15]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [16]:
zero_shot_agent("Where can we see the Eiffel tower?")

In [17]:
zero_shot_agent("Renson has 3 books. He borrows 5 more books from the library. After a week he returns 2 books. How many books does he have now?")

In [18]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm,
)

In [19]:
tools[0].name, tools[0].description

In [20]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent.invoke(prompt=query)
    print(f'Spent a total of {cb.total_tokens} tokens')
    
    return result

In [21]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [22]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False)
)

In [23]:
from sqlalchemy import create_engine, insert

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [24]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [25]:
def insert_obs(obs):
    stmt = insert(stocks).values(
        obs_id=obs[0],
        stock_ticker=obs[1],
        price=obs[2],
        date=obs[3]
    )
    with engine.begin() as conn:
        conn.execute(stmt)

In [26]:
for obs in observations:
    insert_obs(obs)

In [27]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [28]:
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [29]:
from langchain.agents import Tool

sql_tool = Tool(
    name = "Stock DB",
    func = sql_chain.run,
    description="Useful when you need to answer questions about stocks and their price."
)

In [30]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [31]:
tools.append(sql_tool)

In [32]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [33]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)

In [34]:
def count_tokens(chain, query):
    # Assuming 'chain' has a method 'invoke' that requires a parameter 'input'
    result = chain.invoke(input=query)
    # return len(result.split())  # Example of processing result
    return len(result)

In [35]:
result = count_tokens(
    zero_shot_agent,
"What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' on January 3rd and the ratio between the same stock prices on January 4th?"
)

In [36]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

In [ ]:
#conversational react

In [37]:
tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [38]:
tools.append(sql_tool)

In [39]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

In [40]:
conversational_agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

In [42]:
result = count_tokens(
    conversational_agent,
    "Please provide me the stock prices for ABC and XYZ on January 1st"
)

In [43]:
result = count_tokens(
    conversational_agent,
    "Please provide me the ratio of stock prices for ABC and XYZ on January 1st"
)

In [44]:
print(conversational_agent.agent.llm_chain.prompt.template)

In [45]:
result = count_tokens(
    conversational_agent,
    "What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' on January 3rd and the ratio between the same stock prices on January 4th."
    
)

In [ ]:
#React docstore

In [47]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia' 
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia' 
    )
]

In [48]:
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [50]:
count_tokens(docstore_agent, "Who was Duke of Marlborough?")

In [55]:
import os

In [58]:
from langchain_openai import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

llm = OpenAI(temperature=0)
search = SerpAPIWrapper(serpapi_api_key = os.getenv("SERP_API_KEY"))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='Google Search'
    )
]
                        

In [59]:
self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True) 

In [60]:
print(self_ask_with_search.agent.llm_chain.prompt.template)